In [ ]:
import globus_automate_client
import mdf_toolbox
import json
import time
from deriva.core import DerivaServer

In [ ]:
from getpass import getpass
smtp_user = getpass("SMTP Username: ")
smtp_pass = getpass("SMTP Password: ")
smtp_hostname = "email-smtp.us-east-1.amazonaws.com"

In [ ]:
native_app_id = "417301b1-5101-456a-8a27-423e71a2ae26"  # Premade native app ID
flows_client = globus_automate_client.create_flows_client(native_app_id)

In [ ]:
deriva_ap_url = "https://ap-dev.nih-cfde.org/"
deriva_ap_scope = "https://auth.globus.org/scopes/21017803-059f-4a9b-b64c-051ab7c1d05d/demo"

sender_email = "jgaff@uchicago.edu"
admin_email = "jgaff@uchicago.edu"

In [ ]:
failure_text = ("'Your CFDE submission (' + action_id + ') failed to ingest into DERIVA "
                "with this error:\\n' + error")

success_email_template = ("Your CFDE submission ($action_id) has been successfully ingested, "
                          "and can be viewed here: $catalog_link \n Thank you.")

test_sub_success_template = ("Your test CFDE submission ($action_id) was successfully tested with DERIVA. "
                             "No errors were encountered.")

In [ ]:
full_submission_flow_def = {
    "api_version": "1.0",
    "definition": {
        "StartAt": "ChooseTransfer",
        "States": {
            "ChooseTransfer": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.source_endpoint_id",
                    "BooleanEquals": False,
                    "Next": "CreateNoTransferVars"
                }],
                "Default": "TransferData"
            },
            "TransferData": {
                "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
                "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "source_endpoint_id.$": "$.source_endpoint_id",
                    "destination_endpoint_id.$": "$.cfde_ep_id",
                    "transfer_items": [{
                        "source_path.$": "$.source_path",
                        "destination_path.$": "$.cfde_ep_path",
                        "recursive.$": "$.is_directory"
                    }]
                },
                "ResultPath": "$.TransferResult",
                "Type": "Action",
                "WaitTime": 86400,
                "Next": "CreateTransferVars"
            },
            "CreateTransferVars": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "base_url + dest_path",
                        "arguments": {
                            "base_url.$": "$.cfde_ep_url",
                            "dest_path.$": "$.cfde_ep_path"
                        },
                        "result_path": "data_url"
                    }]
                },
                "ResultPath": "$.data_url",
                "WaitTime": 86400,
                "Catch": [{
                  "ErrorEquals": ["States.ALL"],
                  "Next": "ErrorState"
                }],
                "Next": "DerivaIngest"
            },
            "CreateNoTransferVars": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "data_url",
                        "arguments": {
                            "data_url.$": "$.data_url"
                        },
                        "result_path": "data_url"
                    }]
                },
                "ResultPath": "$.data_url",
                "WaitTime": 86400,
                "Catch": [{
                  "ErrorEquals": ["States.ALL"],
                  "Next": "ErrorState"
                }],
                "Next": "DerivaIngest"
            },
            "DerivaIngest": {
                "ActionScope": deriva_ap_scope,
                "ActionUrl": deriva_ap_url,
                #"ExceptionOnActionFailure": True,
                #TODO
                # Require new catalog, set ACL to author only
                "Parameters": {
                    "data_url.$": "$.data_url.details.data_url",
                    "operation": "ingest",
                    "globus_ep.$": "$.cfde_ep_id",
                    # "server": "demo.derivacloud.org",
                    # "catalog_id.$": ,
                    "dcc_id.$": "$.dcc_id",
                    "test_sub.$": "$.test_sub"
                },
                "ResultPath": "$.DerivaIngestResult",
                "Type": "Action",
                "WaitTime": 86400,
                "Catch": [{
                  "ErrorEquals": ["States.ALL"],
                  "Next": "ErrorState"
                }],
                "Next": "CheckDerivaIngest"
            },
            "CheckDerivaIngest": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.DerivaIngestResult.details.error",
                    "BooleanEquals": False,
                    "Next": "CheckTestSub"
                }],
                "Default": "FailDerivaIngest"
            },
            "CheckTestSub": {
                "Type": "Choice",
                "Choices": [{
                    "Variable": "$.test_sub",
                    "BooleanEquals": True,
                    "Next": "TestSubEmail"
                }],
                "Default": "EmailSuccess"
            },
            "EmailSuccess": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/notification/notify",
                "ActionScope": "https://auth.globus.org/scopes/helloworld.actions.automate.globus.org/notification_notify",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    # "body_mimetype": "",
                    "body_template": success_email_template,
                    "body_variables": {
                        "action_id.$": "$._context.action_id",
                        "catalog_link.$": "$.DerivaIngestResult.details.deriva_link"
                    },
                    "destination.$": "$._context.email",
                    # "notification_method": "",
                    # "notification_priority": "low",
                    "send_credentials": [{
                        # "credential_method": "",
                        "credential_type": "smtp",
                        "credential_value": {
                            "hostname": smtp_hostname,
                            "username": smtp_user,
                            "password": smtp_pass
                        }
                    }],
                    "__Private_Parameters": ["send_credentials"],
                    "sender": sender_email,
                    "subject": "Submission Succeeded"
                },
                "ResultPath": "$.EmailSuccessResult",
                "WaitTime": 86400,
                "Catch": [{
                  "ErrorEquals": ["States.ALL"],
                  "Next": "ErrorState"
                }],
                "Next": "FlowSuccess"
            },
            "FlowSuccess": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "catalog_link",
                        "arguments": {
                            "catalog_link.$": "$.DerivaIngestResult.details.deriva_link"
                        },
                        "result_path": "deriva_link"
                    }, {
                        "expression": "catalog_id",
                        "arguments": {
                            "catalog_id.$": "$.DerivaIngestResult.details.deriva_id"
                        },
                        "result_path": "deriva_id"
                    }, {
                        "expression": ("'Submission Flow succeeded. Your catalog ID is ' + str(catalog_id) + "
                                       "', and your submission can be viewed at this link: ' + catalog_link"),
                        "arguments": {
                            "catalog_link.$": "$.DerivaIngestResult.details.deriva_link",
                            "catalog_id.$": "$.DerivaIngestResult.details.deriva_id"
                        },
                        "result_path": "message"
                    }]
                },
                "ResultPath": "$.SuccessState",
                "WaitTime": 86400,
                "Catch": [{
                  "ErrorEquals": ["States.ALL"],
                  "Next": "ErrorState"
                }],
                "Next": "FinishFlow"
            },
            "TestSubEmail": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/notification/notify",
                "ActionScope": "https://auth.globus.org/scopes/helloworld.actions.automate.globus.org/notification_notify",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    # "body_mimetype": "",
                    "body_template": test_sub_success_template,
                    "body_variables": {
                        "action_id.$": "$._context.action_id"
                    },
                    "destination.$": "$._context.email",
                    # "notification_method": "",
                    # "notification_priority": "low",
                    "send_credentials": [{
                        # "credential_method": "",
                        "credential_type": "smtp",
                        "credential_value": {
                            "hostname": smtp_hostname,
                            "username": smtp_user,
                            "password": smtp_pass
                        }
                    }],
                    "__Private_Parameters": ["send_credentials"],
                    "sender": sender_email,
                    "subject": "Test Submission Succeeded"
                },
                "ResultPath": "$.TestSubSuccessResult",
                "WaitTime": 86400,
                "Catch": [{
                  "ErrorEquals": ["States.ALL"],
                  "Next": "ErrorState"
                }],
                "Next": "TestSubSuccess"
            },
            "TestSubSuccess": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "catalog_link",
                        "arguments": {
                            "catalog_link": "None"
                        },
                        "result_path": "deriva_link"
                    }, {
                        "expression": "catalog_id",
                        "arguments": {
                            "catalog_id": "None"
                        },
                        "result_path": "deriva_id"
                    }, {
                        "expression": "'Test Submission Flow succeeded. No DERIVA errors were encountered.'",
                        "result_path": "message"
                    }]
                },
                "ResultPath": "$.SuccessState",
                "WaitTime": 86400,
                "Catch": [{
                  "ErrorEquals": ["States.ALL"],
                  "Next": "ErrorState"
                }],
                "Next": "FinishFlow"
            },
            "FailDerivaIngest": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": failure_text,
                        "arguments": {
                            "action_id.$": "$._context.action_id",
                            "error.$": "$.DerivaIngestResult.details.error"
                        },
                        "result_path": "error"
                    }]
                },
                "ResultPath": "$.FailureState",
                "WaitTime": 86400,
                "Catch": [{
                  "ErrorEquals": ["States.ALL"],
                  "Next": "ErrorState"
                }],
                "Next": "FailDerivaIngest2"
            },
            "FailDerivaIngest2": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/notification/notify",
                "ActionScope": "https://auth.globus.org/scopes/helloworld.actions.automate.globus.org/notification_notify",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    # "body_mimetype": "",
                    "body_template.$": "$.FailureState.details.error",
                    "destination.$": "$._context.email",
                    # "notification_method": "",
                    # "notification_priority": "low",
                    "send_credentials": [{
                        # "credential_method": "",
                        "credential_type": "smtp",
                        "credential_value": {
                            "hostname": smtp_hostname,
                            "username": smtp_user,
                            "password": smtp_pass
                        }
                    }],
                    "__Private_Parameters": ["send_credentials"],
                    "sender": sender_email,
                    "subject": "Submission Failed to Ingest"
                },
                "ResultPath": "$.FailDerivaIngestResult",
                "WaitTime": 86400,
                "Catch": [{
                  "ErrorEquals": ["States.ALL"],
                  "Next": "ErrorState"
                }],
                "Next": "FinishFlow"
            },
            "ErrorState": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/notification/notify",
                "ActionScope": "https://auth.globus.org/scopes/helloworld.actions.automate.globus.org/notification_notify",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    # "body_mimetype": "",
                    "body_template.=": ("A CFDE Flow has errored. Please check the log for this Flow:\n"
                                        "Flow instance ID: `$._context.action_id`.\n\nNOTE: The catalog "
                                        "for this submission has not been deleted, to aid debugging. "
                                        "Please manually delete the catalog when convenient."),
                    "destination": admin_email,
                    # "notification_method": "",
                    # "notification_priority": "low",
                    "send_credentials": [{
                        # "credential_method": "",
                        "credential_type": "smtp",
                        "credential_value": {
                            "hostname": smtp_hostname,
                            "username": smtp_user,
                            "password": smtp_pass
                        }
                    }],
                    "__Private_Parameters": ["send_credentials"],
                    "sender": sender_email,
                    "subject": "Submission Failed to Ingest"
                },
                "ResultPath": "$.ErrorStateResult",
                "WaitTime": 86400,
                "Next": "ErrorFlowLog"
            },
            "ErrorFlowLog": {
                "Type": "Action",
                "ActionUrl": "https://actions.globus.org/expression_eval",
                "ActionScope": "https://auth.globus.org/scopes/5fac2e64-c734-4e6b-90ea-ff12ddbf9653/expression",
                "ExceptionOnActionFailure": True,
                "Parameters": {
                    "expressions": [{
                        "expression": "text",
                        "arguments": {
                            "text": ("A service error has occurred, and the CFDE team has been notified. "
                                     "You may be contacted with additional details.")
                        },
                        "result_path": "error"
                    }]
                },
                "ResultPath": "$.ErrorState",
                "WaitTime": 86400,
                "Next": "FinishFlow"
            },
            "FinishFlow": {
                "Type": "Pass",
                "End": True
            },
        }
    },
    "description": ("Run the CFDE submission flow."),
    "runnable_by": ["urn:globus:groups:id:a437abe3-c9a4-11e9-b441-0efb3ba9a670"],  # CFDE DERIVA Demo
    "synchronous": False,
    "title": "CFDE Submission",
    "types": ["Action", "Choice"],
    "visible_to": ["urn:globus:groups:id:a437abe3-c9a4-11e9-b441-0efb3ba9a670"]  # CFDE DERIVA Demo
}
input_schema = {
    # For Transfer from user (optional fields required if using Globus Transfer)
    "source_endpoint_id": "str or False",
    "source_path": "str, optional",
    "cfde_ep_id": "str or False/None",
    "cfde_ep_path": "str, optional",
    "cfde_ep_url": "str, optional",
    "is_directory": "str, optional",
    # For HTTP download (required if not using Globus Transfer)
    "data_url": "str, optional"
}
pass

In [ ]:
full_flow_deploy_res = flows_client.deploy_flow(
    flow_definition=full_submission_flow_def["definition"],
    title=full_submission_flow_def["title"],
    description=full_submission_flow_def["description"],
    visible_to=full_submission_flow_def["visible_to"],
    runnable_by=full_submission_flow_def["runnable_by"],
)
full_flow_id = full_flow_deploy_res["id"]
full_flow_scope = full_flow_deploy_res["globus_auth_scope"]

In [ ]:
print(full_flow_id)
print(full_flow_scope)

In [ ]:
# Updating Flows causes errors - recommend redeploy instead
'''
full_flow_update_res = flows_client.update_flow(
    flow_id=full_flow_id,
    flow_definition=full_curation_flow_def["definition"],
    title=full_curation_flow_def["title"],
    description=full_curation_flow_def["description"],
    visible_to=full_curation_flow_def["visible_to"],
    runnable_by=full_curation_flow_def["runnable_by"],
)
'''

In [ ]:
#print(json.dumps(flows_client.get_flow(full_flow_id).data, indent=4, sort_keys=True))

In [ ]:
full_flow_input = {
    # For Transfer from user
    "source_endpoint_id": False,
    #"source_path": "str, optional",
    #"fair_re_path": "str, optional",
    #"is_directory": "str, optional",
    # For HTTP download
    # Bad data
    #"data_url": "https://317ec.36fe.dn.glob.us/public/CFDE/ingest_bags/hmp_noaux_170420.tgz",
    # Good data
    "data_url": "https://317ec.36fe.dn.glob.us/public/CFDE/metadata/datapackage_9ffed73c7ad3a52de17e467a8a5024af99d0ed84.zip",
    # For curation
    "author_email": "jgaff@uchicago.edu"
}
pass

In [ ]:
full_flow_res = flows_client.run_flow(full_flow_id, full_flow_scope, full_flow_input)
full_flow_res.data

In [ ]:
status = flows_client.flow_action_status(full_flow_id, full_flow_scope, full_flow_res["action_id"]).data
print(json.dumps(status, indent=4, sort_keys=True))

In [ ]:
flows_client.flow_action_log(full_flow_id, full_flow_scope, full_flow_res["action_id"], limit=100).data

In [ ]:
status = flows_client.flow_action_status(full_flow_id, full_flow_scope, "72a62c9f-dad6-4bc7-b3ef-1b37c5bb800c").data
print(json.dumps(status, indent=4, sort_keys=True))

In [ ]:
flow_id = "7e94c079-fc4c-4b11-85dc-349301b7e607"
instance_id = "fbd15d5d-7ce7-409b-ab6d-935a01cac55f"
flow_scope = flows_client.get_flow(flow_id)["globus_auth_scope"]
flows_client.flow_action_log(flow_id,
                             flow_scope,
                             instance_id,
                             limit=100).data

In [ ]:
flows_client.get_flow(flow_id).data

In [ ]:
deriva_scope = "https://auth.globus.org/scopes/app.nih-cfde.org/deriva_all"
deriva_auth = mdf_toolbox.login(deriva_scope)[deriva_scope]
deriva_creds = {"bearer-token": deriva_auth.access_token}

In [ ]:
servername = "app-dev.nih-cfde.org"
server = DerivaServer("https", servername, deriva_creds)

In [ ]:
catalog_id = 36
catalog = server.connect_ermrest(catalog_id)

In [ ]:
delete_res = catalog.delete_ermrest_catalog(really=True)
print(delete_res)